In [1]:
import requests
import cv2
import numpy as np
import os
import random
import os
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
import requests
import wandb
from sklearn.metrics import f1_score
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch.nn.functional as F
from transformers import SegformerImageProcessor, SegformerForSemanticSegmentation,UperNetForSemanticSegmentation
from glob import glob
import cv2
from torch.utils.data import ConcatDataset, DataLoader

c:\ProgramData\Anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [14]:
files = glob("groundtruth/*.png")

In [11]:
bad = 0

#find files where less than 1% of the pixels are white
for file in files:
    orig_file = file.replace("groundtruth", "images").replace("png", "jpg")
    img = Image.open(file).convert('RGB')
    img = np.array(img)
    frac = np.sum(img == 255) / (img.shape[0] * img.shape[1] * img.shape[2])
    if frac < 0.02:
        print(file + ": " + str(frac))
        bad+=1
        # img_orig = Image.open(orig_file).convert('RGB')
        # fig, ax = plt.subplots(1, 2, figsize=(10, 10))
        # ax[0].imshow(img_orig)
        # ax[1].imshow(img)
        # plt.show()


print("bad: " + str(bad))

groundtruth\boston_10003.png: 0.00275
groundtruth\boston_10004.png: 0.0
groundtruth\boston_10010.png: 0.004575
groundtruth\boston_10015.png: 0.008275
groundtruth\boston_10016.png: 0.00731875
groundtruth\boston_10017.png: 0.00478125
groundtruth\boston_10023.png: 0.00245
groundtruth\boston_10071.png: 0.00625625
groundtruth\boston_10072.png: 0.0
groundtruth\boston_10073.png: 0.0
groundtruth\boston_10074.png: 0.0
groundtruth\boston_10075.png: 0.0
groundtruth\boston_10076.png: 0.0
groundtruth\boston_10077.png: 0.0
groundtruth\boston_10078.png: 0.0
groundtruth\boston_10079.png: 0.0
groundtruth\boston_10080.png: 0.0
groundtruth\boston_10081.png: 0.0
groundtruth\boston_10084.png: 0.0
groundtruth\boston_10090.png: 0.0
groundtruth\boston_10091.png: 0.0
groundtruth\boston_101.png: 0.0058875
groundtruth\boston_1010.png: 0.0
groundtruth\boston_10111.png: 0.0
groundtruth\boston_10112.png: 0.0
groundtruth\boston_10113.png: 0.0
groundtruth\boston_10119.png: 0.00033125
groundtruth\boston_10120.png: 0.0

In [12]:
print("good: " + str(len(files) - bad))

good: 18786


In [15]:
#delete files where less than 1% of the pixels are white
for file in files:
    orig_file = file.replace("groundtruth", "images").replace("png", "jpg")
    img = Image.open(file).convert('RGB')
    img = np.array(img)
    frac = np.sum(img == 255) / (img.shape[0] * img.shape[1] * img.shape[2])
    if frac < 0.02:
        os.remove(file)
        os.remove(orig_file)